In [1]:
import re
import string
import pandas as pd
import nltk
import itertools
from gensim.models import Word2Vec
import gensim.downloader as api
import numpy as np
corpus = api.load('text8')

In [2]:
df = pd.read_csv('dataset/hacker_news_sample.csv')
df = df.dropna(subset=['text'])
# remove html symbols
df['text'] = df['text'].apply(lambda row: re.sub('\<[^>]*\>', ' ', row))
df['text'] = df['text'].apply(lambda row: re.sub('\&[^;]*\;', ' ', row))

df['text'] = df['text'].apply(lambda row: re.sub('\d', '', row).lower())
df['text'] = df['text'].apply(lambda row: row.translate(str.maketrans('', '', string.punctuation)))
df['tokenized'] = df['text'].apply(lambda row: nltk.word_tokenize(row))

print(df['tokenized'][0])

['which', 'leads', 'me', 'to', 'say', 'why', 'are', 'you', 'using', 'c', 'to', 'do', 'x', 'because', 'they', 'know', 'c', 'it', 's', 'fast', 'and', 'it', 'has', 'lots', 'of', 'libs', 'available', 'they', 'might', 'also', 'dislike', 'java', 'or', 'cl', 'not', 'every', 'engineering', 'decision', 'is', 'perfect', 'lots', 'of', 'factors', 'play', 'in', 'attempts', 'to', 'combine', 'the', 'best', 'of', 'c', 'speed', 'with', 'the', 'best', 'of', 'scripting', 'languages', 'easy', 'to', 'do', 'things', 'fast', 'without', 'having', 'to', 'pay', 'attention', 'to', 'what', 'you', 'are', 'doing', 'in', 'my', 'opinion', 'end', 'up', 'merely', 'joining', 'the', 'worst', 'of', 'both', 'worlds', 'rather', 'than', 'the', 'best', 'of', 'both', 'worlds', 'the', 'pay', 'attention', 'things', 'is', 'to', 'needless', 'complexity', 'memory', 'management', 'etc', 'they', 'only', 'reason', 'we', 'put', 'up', 'with', 'those', 'things', 'was', 'to', 'get', 'speed', 'if', 'we', 'can', 'get', 'adequate', 'speed', 

1. O objetivo dessa questão é desenvolver um buscador de documentos.\n
a) escolha e aplique um modelo do tipo word2vec a seus textos, compatível com o idioma de seus textos (inglês ou português).

In [3]:
w2v = Word2Vec(df['tokenized'])
model = Word2Vec(corpus)

b) escolha 5 palavras de consulta que não estejam em nenhum dos textos. Para cada palavra de consulta, encontre as 3 palavras de seu conjunto de textos mais parecidas com cada uma das palavras de consulta e exiba os documentos onde estas palavras aparecem.

In [4]:
words = ['bottleneck', 'draconian', 'jakarta', 'anachronism', 'appoint']
print(len(list(itertools.chain(*df['tokenized'].tolist()))))
for word in words:
    if word not in list(itertools.chain(*df['tokenized'].tolist())):
        print(f'word "{word}" not in dataset')

for word in words:
    similarities = []
    for word2 in list(itertools.chain(*df['tokenized'].tolist())):
        if word2 in w2v.wv:
            vec1 = w2v.wv[word2]
            vec2 = model.wv[word]
            similarities.append((word2, np.sum(vec1*vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))))
    print(word, '->', sorted(list(dict(similarities).items()), key=lambda x: x[1], reverse=True)[:3])



507363
word "draconian" not in dataset
word "jakarta" not in dataset
word "anachronism" not in dataset
word "appoint" not in dataset
bottleneck -> [('prediction', 0.35406077), ('gon', 0.34344906), ('struck', 0.34233242)]
draconian -> [('matters', 0.62689024), ('accomplished', 0.6255134), ('opposition', 0.62537265)]
jakarta -> [('engagement', 0.43495196), ('credible', 0.42365366), ('warrant', 0.4232228)]
anachronism -> [('frustration', 0.30974063), ('casually', 0.30972105), ('overnight', 0.3063045)]
appoint -> [('out', 0.2260439), ('use', 0.2211654), ('give', 0.21638574)]


c) Seja d um documento da base e w uma palavra de consulta. Implemente o seguinte algoritmo para buscar documentos:
1. Encontre d10(w): a lista com as 10 palavras mais parecidas com w em um certo documento d.
2. Para cada documento d, calcule a distância média DM10(w) entre w e as palavras de d10(w).
3. Recupere os 3 documentos da base cuja DM10(w)  ́e menor.